# Hash Relational Tensor (HRT) Demo

Interactive exploration of the HRT extension to Manifold OS.

In [1]:
# Import HRT components
from core.hrt import (
    HRT, HRTConfig, HRTFactory,
    AdjacencyMatrix, HLLSetLattice,
    BasicHLLSet, Cover, SpecialToken
)
from core.manifold_os import ManifoldOS
from core.kernel import Kernel, HLLSet

import numpy as np

print("HRT components imported successfully!")

HRT components imported successfully!


## Part 1: HRT Configuration

In [2]:
# Create configuration
config = HRTConfig(p_bits=8, h_bits=16)

print(f"P bits: {config.p_bits}")
print(f"H bits: {config.h_bits}")
print(f"Dimension: {config.dimension}")
print(f"Basic HLLSets: {config.num_basic_hllsets}")

P bits: 8
H bits: 16
Dimension: 4098
Basic HLLSets: 8196


## Part 2: Adjacency Matrix

In [3]:
am = AdjacencyMatrix(config=config)
print(f"AM shape: {am.matrix.shape}")

# Add relationships
am = am.update(row=5, col=10, value=0.8)
am = am.update(row=5, col=11, value=0.6)

print(f"Non-zero: {len(am.get_nonzero_entries())}")

AM shape: (4098, 4098)
Non-zero: 2


## Part 3: HLLSet Lattice

In [4]:
lattice = HLLSetLattice(config=config)
print(f"Lattice: {lattice}")

r_5 = lattice.get_row_basic(5)
c_10 = lattice.get_col_basic(10)

print(f"r_5: {r_5}")
print(f"c_10: {c_10}")

Lattice: Lattice(4098 rows, 4098 cols)
r_5: Basic(r_5)
c_10: Basic(c_10)


## Part 4: Complete HRT

In [5]:
factory = HRTFactory(config)

perceptron_data = {
    "visual": {"red", "green", "blue"},
    "audio": {"low", "mid", "high"},
}

kernel = Kernel()
hrt = factory.create_from_perceptrons(perceptron_data, kernel)

print(f"HRT: {hrt.name[:20]}...")
print(f"AM non-zero: {len(hrt.get_am().get_nonzero_entries())}")

HRT: 3aa3e9f2ee9e1e96f292...
AM non-zero: 6


## Part 5: HRT in Manifold OS

In [6]:
mos = ManifoldOS(hrt_config=config)

mos.add_perceptron("p1", "camera")
mos.add_perceptron("p2", "microphone")

state = mos.process_cycle({"p1": {"a", "b"}, "p2": {"c", "d"}})

print(f"State: {state.state_hash[:16]}...")
print(f"HRT: {state.hrt_hash[:16]}...")

State: ...
HRT: 3aa3e9f2ee9e1e96...


## Part 6: Projections

In [7]:
future = mos.project_future([10, 20, 30])
past = mos.project_past([5, 15, 25])

print(f"Future shape: {future.shape}")
print(f"Past shape: {past.shape}")
print(f"Future max: {future.max():.3f}")
print(f"Past max: {past.max():.3f}")

Future shape: (4098,)
Past shape: (4098,)
Future max: 0.000
Past max: 0.000


In [8]:
# Stats
stats = mos.stats()
print("HRT Stats:")
for k, v in stats.items():
    if 'hrt' in k.lower():
        print(f"  {k}: {v}")

HRT Stats:
  persistent_hrts: 1
  hrt_name: 3aa3e9f2ee9e1e96
  hrt_dimension: 4098
  hrt_am_nonzero: 4
  hrt_covers_cached: 1
